In [137]:
# import time
# time.sleep(1800)

---

### **3. Calculo de Pobreza**
  - **3.1. Carga y ajuste de datos:** 
    - **3.1.1.** Carga de datos monetarios.
    - **3.1.2.** Ajuste usando el IPC.
    - **3.1.3.** Calculo de la canasta básica.
    - **3.1.4.** Calculo de la línea de pobreza.
    - **3.1.5.** Calculo de la línea de indigencia.



In [138]:
# 1. Módulos y Bibliotecas
# Importar todos los módulos y bibliotecas necesarios.
import pandas as pd
import os
from funciones import personas_ingresos_por_trimestre, ingresos_a_metricas_pobreza, P0910, geo_hogares, generate_Qs_from_year


In [139]:
# 2. Configuración de Directorios y Parámetros
# Establecer y verificar las rutas de los directorios.
# Definir los parámetros necesarios.
if not os.path.exists('./../data/Pobreza/'):
    os.makedirs('./../data/Pobreza/')

# -------------------
# Parameters and Configuration
# -------------------

FRAC = 0.02
START_YEAR = 2023
END_YEAR = 2024
EXPERIMENT_TAG = 'ARG'

PATH_POBREZA = './../data/Pobreza/'



In [140]:
# 3. Exploración y Preprocesamiento de Datos
# Cargar conjuntos de datos.
# Explorar datos iniciales.
# Preparar datos para síntesis.

# Carga de Datos Auxiliares: Datos del adulto equivalente y Canasta básica regional deflactada
ad_eq = pd.read_csv('./../data/info/adulto_eq.csv')
url_canasta_q_deflac = 'https://raw.githubusercontent.com/matuteiglesias/canastasINDEC/main/data/CB_Reg_defl_Q.csv'
CB_ipc = pd.read_csv(url_canasta_q_deflac)
print(CB_ipc.tail())

# Preparación de la Información Geográfica
radio_ref = pd.read_csv('./../data/info/radio_ref.csv', usecols = ['RADIO_REF_ID', 'DPTO', 'FRAC_REF_ID', 'NOMDPTO', 'radio'])
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)
radio_ref['COD_2010'] = radio_ref['radio'].astype(str).str.zfill(9)
radio_ref = radio_ref.drop('radio', axis = 1)
radio_ref_cols = radio_ref

claves_dptos = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/elecciones-ARG/main/datos/BD/claves_dptos_ref.csv')
claves_dptos_cols = claves_dptos[['distrito_id', 'seccion_id', 'IN1', 'NAM']].drop_duplicates()
display('dtypes claves_dptos_ref', claves_dptos_cols.dtypes)

radios_circuitos_secciones_ref = pd.read_csv('./../../CNE-INDEC-georef/info/radios_circuitos_secciones_ref.csv')
radios_circuitos_secciones_ref = radios_circuitos_secciones_ref[['COD_2010', 'distrito_id', 'seccion_id', 'seccion_nombre', 'circuito']]
display('radios_circuitos_secciones_ref', radios_circuitos_secciones_ref.max())

DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()


              Q             Region     CBA     CBT
505  2024-02-15  gran_buenos_aires 1401.77 3355.03
506  2024-02-15            noreste 1254.71 2811.27
507  2024-02-15           noroeste 1217.88 2705.13
508  2024-02-15           pampeana 1389.22 3323.90
509  2024-02-15          patagonia 1448.20 3934.47


'dtypes claves_dptos_ref'

distrito_id    float64
seccion_id     float64
IN1            float64
NAM             object
dtype: object

'radios_circuitos_secciones_ref'

COD_2010          940140813
distrito_id              24
seccion_id              134
seccion_nombre      Ñorquín
circuito          zonaincon
dtype: object

In [141]:
# DPTO_Region.head()

# DPTO	Region
# 0	2001	gran_buenos_aires
# 329	2002	gran_buenos_aires
# 527	2003	gran_buenos_aires
# 781	2004	gran_buenos_aires
# 1033	2005	gran_buenos_aires


##  **Procesamiento por Año y Trimestre**


    - Bucle por años
        - Carga de la población sintética (muestra del censo) por año
        - Cómputo de atributos geográficos
        - Procesamiento del nivel educativo
        - Bucle por trimestre
            - Fusión de la población sintética con datos de ingresos
            - Cálculo de métricas de pobreza para hogares

In [142]:
## pd format to 2 decimals
pd.options.display.float_format = '{:.2f}'.format

In [143]:
# experiment_tag = 'ARG'
# frac = 0.01
# Q = '2016-08-15'

In [144]:
# filename = f'/media/matias/Elements/suite/resultados/RFC4_{str(FRAC)}_{Q}_{EXPERIMENT_TAG}.csv'

# df = pd.read_csv(filename, nrows = 5)

In [145]:


# Loop through the years
for yr in range(START_YEAR, END_YEAR):
    yr = str(yr)  # Convert year to string for consistency with quarters
    print(f"Processing data for year {yr}...")

    # Carga de la población sintética (muestra del censo) para el año especificado.
    # Nota: La ruta especificada podría estar apuntando a un disco externo.
    poblacion_synth_file = f'/media/matias/Elements/suite/poblaciones/table_f{str(FRAC)}_{yr}_{EXPERIMENT_TAG}.csv'

    columns_table = ['ID', 'RADIO_REF_ID', 'HOGAR_REF_ID', 'AGLOMERADO']
    poblacion_yr = pd.read_csv(poblacion_synth_file, usecols=columns_table)
    poblacion_yr['ANO4'] = int(yr)

    # Cómputo opcional de atributos geográficos.
    # Si el archivo ya existe y `overwrite_geo` está establecido en True, 
    # se sobrescribirá el archivo existente con nuevos datos.
    hogares_geo_file = f'./../data/Pobreza/geo_households_sample{str(FRAC)}_{yr}_{EXPERIMENT_TAG}.csv'

    overwrite_geo = True

    if not os.path.exists(hogares_geo_file) or overwrite_geo:
        muestra_hogares = poblacion_yr[['RADIO_REF_ID', 'AGLOMERADO', 'HOGAR_REF_ID']].drop_duplicates()
        hogares_geo = geo_hogares(muestra_hogares, radio_ref_cols, radios_circuitos_secciones_ref, claves_dptos_cols, FRAC)
        hogares_geo.to_csv(hogares_geo_file, index=False)
        print(f"{hogares_geo_file} saved for persons")

    # Load synthetic population for the year
    # poblacion_synth_file = ...
    columns_table = ['ID', 'RADIO_REF_ID', 'AGLOMERADO', 'DPTO', 'HOGAR_REF_ID', 'P02', 'P03', 'P09', 'P10']
    poblacion_yr = pd.read_csv(poblacion_synth_file, usecols=columns_table)
    poblacion_yr['ANO4'] = int(yr)

    # Nivel Educativo
    poblacion_yr = P0910(poblacion_yr)
    print(f"size/FRAC = Pob after processing educational level: {len(poblacion_yr)/FRAC}.")

    # Procesamiento de datos para cada trimestre del año en cuestión.
    # relevant_quarters = [q for q in quarters if q.startswith(yr)]
    relevant_quarters = generate_Qs_from_year(yr)
    for Q in relevant_quarters:
        print(f"Processing data for quarter {Q}...")
        
        # Merge synthetic population with income data
        personas_ingresos_Q_df = personas_ingresos_por_trimestre(poblacion_yr, Q, FRAC, experiment_tag='ARG')

        # Check if the result is not None before proceeding
        if personas_ingresos_Q_df is not None:
            personas_ingresos_Q_file = f'./../data/Pobreza/individual_income_sample{str(FRAC)}_{Q}_{EXPERIMENT_TAG}.csv'
            personas_ingresos_Q_df['P47T_persona'] = personas_ingresos_Q_df['P47T_persona'].round(2) # Round income to nearest 10

            personas_ingresos_Q_df[['ID', 'HOGAR_REF_ID', 'Q', 'P47T_persona', 'P02', 'P03', 'P0910']].to_csv(personas_ingresos_Q_file, index=False)        ### GUARDA INGRESOS PERSONAS

            print(f"Poblacion y ingresos para {Q} guardados. Pob aprox: {len(personas_ingresos_Q_df)/FRAC} personas.\n Mean and stats below.")
            display(personas_ingresos_Q_df.P47T_persona.describe())
            display(personas_ingresos_Q_df.select_dtypes(exclude=['object']).mean())

            # Compute poverty metrics for households
            pobreza_hogares_file = f'./../data/Pobreza/household_poverty_sample{str(FRAC)}_q{Q}.csv'
            muestra_hogares = ingresos_a_metricas_pobreza(personas_ingresos_Q_df, ad_eq, DPTO_Region, CB_ipc, FRAC)   ### 
            # def ingresos_a_metricas_pobreza(df_ingresos, ad_eq, DPTO_Region, CB_ipc, frac, columnas_pesos=['P47T_persona']):

            for col in ['P47T_hogar', 'CBA', 'CBT', 'CB_EQUIV', 'gap_pobreza', 'gap_indigencia']:
                muestra_hogares[col] = muestra_hogares[col].round(2)

            muestra_hogares.to_csv(pobreza_hogares_file, index=False)        ### GUARDA POBREZA HOGARES

            print(f"{pobreza_hogares_file} saved for households.\n Mean and stas below.")
            display(muestra_hogares.select_dtypes(exclude=['object']).mean())
            display(muestra_hogares[['CBA', 'CBT', 'CB_EQUIV', 'P47T_hogar']].describe())
        else:
            print(f"Skipped processing for quarter {Q} due to missing or invalid data.")

Processing data for year 2023...


size/frac = Pob after merging with radio_ref_cols: 13860650.0
maxs:  RADIO_REF_ID        52380
AGLOMERADO             93
HOGAR_REF_ID     12197634
FRAC_REF_ID          5426
DPTO                94014
NOMDPTO           Ñorquín
PROV                   94
NOMPROV           Tucumán
Region          patagonia
COD_2010        940140813
dtype: object


RADIO_REF_ID      277213
AGLOMERADO        277213
HOGAR_REF_ID      277213
FRAC_REF_ID       277213
DPTO              277213
NOMDPTO           277213
PROV              277213
NOMPROV           277213
Region            277213
COD_2010          277213
distrito_id       276197
seccion_id        276197
seccion_nombre    276197
circuito          276197
dtype: int64

size/frac = Pob after merging with radios_circuitos_secciones_ref: 13860650.0
size/frac = Pob after merging with IN1: 13911450.0
dtypes:  RADIO_REF_ID        int64
AGLOMERADO          int64
HOGAR_REF_ID        int64
FRAC_REF_ID         int64
DPTO                int64
NOMDPTO            object
PROV                int64
NOMPROV            object
Region             object
COD_2010           object
distrito_id       float64
seccion_id        float64
seccion_nombre     object
circuito           object
IN1               float64
NAM                object
dtype: object
./../data/Pobreza/geo_households_sample0.02_2023_ARG.csv saved for persons
size/FRAC = Pob after processing educational level: 46009750.0.
Processing data for quarter 2023-02-15...
reading regression results at:  /media/matias/Elements/suite/resultados/RFC4_0.02_2023-02-15_ARG.csv
size/frac = Poblacion segun muestra pob con estimacion de ingresos: 46009750.0
Poblacion y ingresos para 2023-02-15 guardados. Pob aprox: 46009750.0 p

count   920195.00
mean         2.30
std          1.82
min          0.00
25%          0.00
50%          3.47
75%          3.83
max          5.81
Name: P47T_persona, dtype: float64

ID             54961593179.08
RADIO_REF_ID         25662.14
DPTO                 30276.94
AGLOMERADO              15.85
HOGAR_REF_ID       6352455.91
P02                      1.51
P03                     32.32
P09                      3.25
P10                      0.65
ANO4                  2023.00
P47T_persona             2.30
dtype: float64

size/frac = Pob, input de ingresos_a_pobreza: 46009750.0
Mean of column P47T_persona: 4110.58669035907
size/frac = Pob after canasta calculation: 46009750.0
size(hogares)/frac = Cantidad de hogares: 13859000.0
Mean of column HOGAR_REF_ID: 6168567.476387185
Mean of column P47T_hogar: 13646.51605287165
Mean of column CBA: 3982.3415199114884
Mean of column CBT: 8822.63009828088
Mean of column CB_EQUIV: 2.6552508478245187
Mean of column Pobreza: 0.28868605238473194
Mean of column Indigencia: 0.0717439930730933
Mean of column gap_pobreza: 4823.885954590769
Mean of column gap_indigencia: 9664.17453296016
./../data/Pobreza/household_poverty_sample0.02_q2023-02-15.csv saved for households.
 Mean and stas below.


HOGAR_REF_ID     6168567.48
P47T_hogar         13646.52
CBA                 3982.34
CBT                 8822.63
CB_EQUIV               2.66
Pobreza                0.29
Indigencia             0.07
gap_pobreza         4823.89
gap_indigencia      9664.17
dtype: float64

,CBA,CBT,CB_EQUIV,P47T_hogar
count,277180.00,277180.00,277180.00,277180.00
mean,3982.34,8822.63,2.66,13646.52
std,4744.84,10519.63,3.11,24023.59
min,847.28,1742.01,0.63,0.00
25%,2326.47,5109.85,1.50,6759.83
50%,3649.13,8074.10,2.44,11222.86
75%,5242.31,11604.57,3.51,17559.02
max,1944564.81,4302835.71,1263.10,9976244.35


Processing data for quarter 2023-05-15...
Error: archivo del modelo /media/matias/Elements/suite/resultados/RFC4_0.02_2023-05-15_ARG.csv no existe. Calcular en notebook 02, o ignorar mensaje si el trimestre 2023-05-15 no tuvo EPH publicada por INDEC.values=
Skipped processing for quarter 2023-05-15 due to missing or invalid data.
Processing data for quarter 2023-08-15...
Error: archivo del modelo /media/matias/Elements/suite/resultados/RFC4_0.02_2023-08-15_ARG.csv no existe. Calcular en notebook 02, o ignorar mensaje si el trimestre 2023-08-15 no tuvo EPH publicada por INDEC.values=
Skipped processing for quarter 2023-08-15 due to missing or invalid data.
Processing data for quarter 2023-11-15...
Error: archivo del modelo /media/matias/Elements/suite/resultados/RFC4_0.02_2023-11-15_ARG.csv no existe. Calcular en notebook 02, o ignorar mensaje si el trimestre 2023-11-15 no tuvo EPH publicada por INDEC.values=
Skipped processing for quarter 2023-11-15 due to missing or invalid data.
